In [59]:
!pip install efficientnet
import keras
import efficientnet.keras as efn
import os
import pandas as pd
import numpy as np
from PIL import Image
import os
import pydicom
from tensorflow.keras.preprocessing import image

In [52]:
base_model = efn.EfficientNetB6(weights='imagenet',include_top=False, input_shape=(224, 224, 3))

In [58]:
import keras.layers as L
layer = base_model.output
layer = L.GlobalAveragePooling2D()(layer)
layer = L.Dense(1024,activation='relu')(layer)
layer = L.Dropout(0.4)(layer)
predictions = L.Dense(1,activation='sigmoid')(layer)
   # predictions = tf.cast(predictions,tf.float32)
model = keras.models.Model(inputs = base_model.input, outputs=predictions)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 528, 528, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 264, 264, 56) 1512        input_5[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 264, 264, 56) 224         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 264, 264, 56) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

__________________________________________________________________________________________________
block6d_drop (FixedDropout)     (None, 17, 17, 344)  0           block6d_project_bn[0][0]         
__________________________________________________________________________________________________
block6d_add (Add)               (None, 17, 17, 344)  0           block6d_drop[0][0]               
                                                                 block6c_add[0][0]                
__________________________________________________________________________________________________
block6e_expand_conv (Conv2D)    (None, 17, 17, 2064) 710016      block6d_add[0][0]                
__________________________________________________________________________________________________
block6e_expand_bn (BatchNormali (None, 17, 17, 2064) 8256        block6e_expand_conv[0][0]        
__________________________________________________________________________________________________
block6e_ex

Trainable params: 43,097,049
Non-trainable params: 224,432
__________________________________________________________________________________________________


In [47]:
for layer in base_model.layers:
    layer.trainble = False

In [49]:
base = "../input/siim-isic-melanoma-classification/"
os.listdir(base)

NameError: name 'os' is not defined

In [ ]:
train = pd.read_csv(base + "train.csv")
print(train.shape)
train.head()

In [ ]:
train['sex']=train['sex'].replace({"male":1,"female":0})
train['anatom_site_general_challenge']=train['anatom_site_general_challenge'].fillna("na")
from sklearn.preprocessing import LabelEncoder
lr=LabelEncoder()
train["anatom_site_general_challenge"]=lr.fit_transform(train["anatom_site_general_challenge"])
train.loc[0,['sex',"anatom_site_general_challenge"]].values.astype("float")

In [ ]:
df_0=train[train['target']==0].sample(100,random_state=42)
df_1=train[train['target']==1].sample(100,random_state=42)
train=pd.concat([df_0,df_1])
train=train.reset_index()
train.shape

In [ ]:
train['sex']=train['sex'].fillna(1)
train['age_approx']=train['age_approx'].fillna(train['age_approx'].mean())
train.isnull().sum()

In [ ]:
def create_training_data():
    path = base + "train/" + train["image_name"] +".dcm"
    path = path
    temp = []
    for i,x in path.iteritems():
        k = pydicom.dcmread(x).pixel_array
        k = image.img_to_array(image.array_to_img(k).resize((224, 224)))
        temp.append(k)
    return temp
X = create_training_data()
X=np.array(X)
X.shape

In [62]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])

In [ ]:
model.fit(X,train['target'].values,batch_size=16,epochs=12)

In [ ]:
model.evaluate(X,train['target'].values)

In [ ]:
test = pd.read_csv(base+"test.csv")
print(test.shape)
test.head()

In [ ]:
def create_test_data():
    path = base + "test/" + test["image_name"] +".dcm"
    temp = []
    for i,x in path.iteritems():
        try:
            k = pydicom.dcmread(x).pixel_array
            k = image.img_to_array(image.array_to_img(k).resize((224, 224)))
            k = k/255.0
            temp.append(k)
        except Exception as e:
            print(i,str(e))
    return temp
data = create_test_data()
data = np.array(data)
data.shape

In [ ]:
test["target"]=model.predict(data)
test[["image_name","target"]].to_csv('submission.csv', index=False)